In [ ]:
#Note this accelerated MD is based on tutorial - https://ambermd.org/tutorials/advanced/tutorial22/index.php

In [10]:
import matplotlib.pyplot as plt
import os
import numpy as np
import math
import subprocess

In [11]:
import os

# Strip out the exported bash functions that break Jupyter subshells
for key in list(os.environ):
    if key.startswith("BASH_FUNC_"):
        del os.environ[key]

print("✅ Cleaned exported bash functions. Subshells will be quiet now.")


✅ Cleaned exported bash functions. Subshells will be quiet now.


In [12]:
# set working directory for all future cells
os.chdir("/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/Accelerated_aMD_Amber")
# and check with bash command pwd - note the exclamation mark at the beginning
!pwd
# or in python
os.getcwd()

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/Accelerated_aMD_Amber


'/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/Accelerated_aMD_Amber'

In [13]:
import os, subprocess

def load_amber_env():
    """Load AmberTools + Amber environment into the Jupyter kernel."""
    bash_cmd = """
    module purge
    module load StdEnvACCRE/2023
    module load gcc/12.3 openmpi/4.1.5 cuda/12.2
    module load ambertools/25.0
    module load scipy-stack/2025a
    module load amber/24
    env
    """
    env_output = subprocess.check_output(["bash", "-c", bash_cmd], text=True)
    for line in env_output.splitlines():
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k] = v
    print("✅ Amber environment loaded into Jupyter kernel.")


In [14]:
load_amber_env()

The following modules were not unloaded:
  (Use "module --force purge" to unload all):

  1) CCconfig        6)  ucx/1.14.1         11) flexiblas/3.3.1
  2) gentoo/2023     7)  libfabric/1.18.0   12) imkl/2023.2.0
  3) gcccore/.12.3   8)  pmix/4.2.4         13) StdEnvACCRE/2023
  4) gcc/12.3        9)  ucc/1.2.0
  5) hwloc/2.9.1     10) openmpi/4.1.5
Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2023b
(required by: plumed/2.9.0, ambertools/25.0)
-------------------------------------------------------------------------------




The following have been reloaded with a version change:
  1) ipykernel/2023b => ipykernel/2025a
  2) scipy-stack/2023b => scipy-stack/2025a



✅ Amber environment loaded into Jupyter kernel.


Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2025a
(required by: amber/24)
-------------------------------------------------------------------------------




Inactive Modules:
  1) arrayfire/3.9.0     2) cudnn/8.9.5.29     3) glfw/3.3.8     4) spdlog/1.11.0

Due to MODULEPATH changes, the following have been reloaded:
  1) ambertools/25.0     2) plumed/2.9.0     3) ucc-cuda/1.2.0     4) ucx-cuda/1.14.1

The following have been reloaded with a version change:
  1) cuda/12.2 => cuda/12.6
  2) cudacore/.12.2.2 => cudacore/.12.6.2
  3) ipykernel/2025a => ipykernel/2023b
  4) nccl/2.18.3 => nccl/2.26.2
  5) scipy-stack/2025a => scipy-stack/2023b



In [15]:
#Compute average total potential energy and torsional (dihedral) energy
!mkdir aMD_params
%cd aMD_params
!process_mdout.perl ../../Production/rep1/PTR_001.out

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
mkdir: cannot create directory ‘aMD_params’: File exists
/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/Accelerated_aMD_Amber/aMD_params
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
Processing sander output file (../../Production/rep1/PTR_001.out)...


/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/ipykernel/2025a/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.EPTOT
Outputing summary_rms.EPTOT
Outputing summary.DENSITY
Outputing summary_avg.DENSITY
Outputing summary_rms.DENSITY
Outputing summary.VOLUME
Outputing summary_avg.VOLUME
Outputing summary_rms.VOLUME
Outputing summary.ESCF
Outputing summary_avg.ESCF
Outputing summary_rms.ESCF


In [17]:
%%bash
# Step 1: Write get_dihed.in to get the dihed.dat 
cat > get_dihed.in <<EOF
parm ../../Minimization/prepinHMR.prmtop
trajin ../../Production/rep1/md_prod1_001.nc
energy DIHED out dihed.dat
EOF

echo "✅ get_dihed.in file written."

# Step 2: Run cpptraj
cpptraj -i get_dihed.in

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


✅ get_dihed.in file written.

CPPTRAJ: Trajectory Analysis. V6.29.13 (AmberTools) OpenMP
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| 2 OpenMP threads available.
| Date/time: 09/15/25 03:43:44
| Available memory: 216.272 GB

INPUT: Reading input from 'get_dihed.in'
  [parm ../../Minimization/prepinHMR.prmtop]
	Reading '../../Minimization/prepinHMR.prmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin ../../Production/rep1/md_prod1_001.nc]
	Reading '../../Production/rep1/md_prod1_001.nc' as Amber NetCDF
  [energy DIHED out dihed.dat]
    ENERGY: Calculating energy for atoms in mask '*'
	Calculating terms: 'Bonds' 'Angles' 'Dihedrals' '1-4 VDW' '1-4 Elec.' 'VDW' 'Elec.' 'Kinetic' 'Total'
	Electrostatics method: Simple
	If forces and velocities present KE will be calculated assuming
	velocities are a half step ahead of forces; if only velocities
	present KE will be calculated assuming velocities are on-step.
	Time step for KE calculation

CalledProcessError: Command 'b'# Step 1: Write get_dihed.in to get the dihed.dat \ncat > get_dihed.in <<EOF\nparm ../../Minimization/prepinHMR.prmtop\ntrajin ../../Production/rep1/md_prod1_001.nc\nenergy DIHED out dihed.dat\nEOF\n\necho "\xe2\x9c\x85 get_dihed.in file written."\n\n# Step 2: Run cpptraj\ncpptraj -i get_dihed.in\n'' died with <Signals.SIGINT: 2>.

In [18]:
%%bash
cat > get_dihed.in <<EOF
parm ../../Minimization/prepinHMR.prmtop
trajin ../../Production/rep1/md_prod1_001.nc 1 last 10
energy dihedral out dihed.dat
EOF

echo "✅ get_dihed.in file written (reading every 10th frame, dihedral term only)."

export OMP_NUM_THREADS=4
cpptraj -i get_dihed.in

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


✅ get_dihed.in file written (reading every 10th frame, dihedral term only).

CPPTRAJ: Trajectory Analysis. V6.29.13 (AmberTools) OpenMP
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| 4 OpenMP threads available.
| Date/time: 09/15/25 03:48:20
| Available memory: 216.206 GB

INPUT: Reading input from 'get_dihed.in'
  [parm ../../Minimization/prepinHMR.prmtop]
	Reading '../../Minimization/prepinHMR.prmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin ../../Production/rep1/md_prod1_001.nc 1 last 10]
	Reading '../../Production/rep1/md_prod1_001.nc' as Amber NetCDF
  [energy dihedral out dihed.dat]
    ENERGY: Calculating energy for atoms in mask '*'
	Calculating terms: 'Dihedrals'
---------- RUN BEGIN -------------------------------------------------

PARAMETER FILES (1 total):
 0: prepinHMR.prmtop, 45940 atoms, 14087 res, box: Truncated octahedron, 13813 mol, 13808 solvent

INPUT TRAJECTORIES (1 total):
 0: 'md_prod1_001.nc' is a NetCDF (N

In [22]:
#Your dihed.dat file contains all energy components per frame — 
#but for aMD we only want the dihedral term, which is the DIHED[dih] column (the third numeric column).

#Get Average Dihedral Energy
!grep -v "#" dihed.dat | awk '{s+=$2; n++} END{print "Average dihedral energy =", s/n, "kcal/mol"}'
!echo -n "EthreshP: E(tot)= "; cat summary_avg.EPTOT

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
Average dihedral energy = 3598.75 kcal/mol
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
EthreshP: E(tot)=   100000.000     -135987.7125


In [23]:
%%bash
cpptraj -p ../../Minimization/prepinHMR.prmtop <<EOF
parminfo
EOF

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'



CPPTRAJ: Trajectory Analysis. V6.29.13 (AmberTools) OpenMP
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| 2 OpenMP threads available.
| Date/time: 09/15/25 03:53:29
| Available memory: 210.722 GB

	Reading '../../Minimization/prepinHMR.prmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
INPUT: Reading input from 'STDIN'
  [parminfo]
	Topology prepinHMR.prmtop contains 45940 atoms.
		Title: default_name
		Original filename: ../../Minimization/prepinHMR.prmtop
		14087 residues.
		13813 molecules.
		45977 bonds (43708 to H, 2269 other).
		8288 angles (5227 with H, 3061 other).
		20111 dihedrals (10288 with H, 9823 other).
		Box: Truncated octahedron
		13808 solvent molecules.
		GB radii set: modified Bondi radii (mbondi)
		Non-bonded parameters are present.
TIME: Total execution time: 0.0541 seconds.
--------------------------------------------------------------------------------
To cite CPPTRAJ use:
Daniel R. Roe and Thomas E. Cheatham, III, "PT

In [24]:
# aMD Parameter Report Generator (Python version)
ETOT = float(input("Enter average total potential energy (E_tot) [kcal/mol]: "))
M = int(input("Enter total number of atoms in system (M): "))
EDIH = float(input("Enter average dihedral energy (E_dih) [kcal/mol]: "))
NRES = int(input("Enter number of protein residues (Nres): "))

ALPHAP = 0.16 * M
ETHRESHP = ETOT + ALPHAP
ETHRESHD = EDIH + 4 * NRES
ALPHAD = (4 * NRES) / 5

print("\n=== aMD Parameter Calculation ===")
print(f"a) EthreshP: E(tot)= {ETOT:.0f} kcal mol-1 + (0.16kcal*mol-1 atom-1 * {M} atoms) = {ETHRESHP:.0f} kcal mol-1\n")
print(f"b) alphaP: Alpha(tot)= (0.16kcal mol-1 atom-1 * {M} atoms) = {ALPHAP:.1f} kcal mol-1\n")
print(f"c) EthreshD: E(dih)={EDIH:.0f} kcal mol-1 + (4kcal mol-1 residue-1 * {NRES} solute residues) = {ETHRESHD:.0f} kcal mol-1\n")
print(f"d) alphaD: Alpha(dih)=(1/5)*(4kcal mol-1 residues-1 * {NRES} solute residues) = {ALPHAD:.1f} kcal mol-1")


Enter average total potential energy (E_tot) [kcal/mol]:  -135987.7125
Enter total number of atoms in system (M):  45940
Enter average dihedral energy (E_dih) [kcal/mol]:  3598.75
Enter number of protein residues (Nres):  275



=== aMD Parameter Calculation ===
a) EthreshP: E(tot)= -135988 kcal mol-1 + (0.16kcal*mol-1 atom-1 * 45940 atoms) = -128637 kcal mol-1

b) alphaP: Alpha(tot)= (0.16kcal mol-1 atom-1 * 45940 atoms) = 7350.4 kcal mol-1

c) EthreshD: E(dih)=3599 kcal mol-1 + (4kcal mol-1 residue-1 * 275 solute residues) = 4699 kcal mol-1

d) alphaD: Alpha(dih)=(1/5)*(4kcal mol-1 residues-1 * 275 solute residues) = 220.0 kcal mol-1


In [32]:
%cd ../
!mkdir Production_aMD
%cd Production_aMD

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/Accelerated_aMD_Amber
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/Accelerated_aMD_Amber/Production_aMD


In [56]:
# Note that Ive changed dt to 0.002 and nstlim = 50000000; total time = 100ns still. Did this since 0.004 was unstable with aMD
#changed ntwx = 5000 (prints co-ordinates every 10ps, ntpr = 10000 - writes energy every 20ps and ntwr = 500000 - writes restart file every 1ns

amd_content = f""" MD explicit solvent heavy atom no rest shake dt 0.002 total is 100 nanoseconds
 &cntrl
  imin = 0, nstlim = 50000000, dt=0.002,
  ntx = 1, irest = 0,
  ntwx = 5000, ioutfm = 1, ntxo = 2, ntpr = 10000, ntwr = 500000,
  iwrap = 1, nscm = 1000,
  ntc = 2, ntf = 2, ntb = 2, cut = 8.0,
  ntt = 3, ig = -1, gamma_ln = 3, temp0 = 310, tempi = 310,
  ntp = 1, taup = 2.0, barostat = 2,
  igb = 0, saltcon = 0.0,
  ntr = 0,
  iamd=3,
  ethreshd={ETHRESHD}, alphad={ALPHAD},
  ethreshp={ETHRESHP}, alphap={ALPHAP},
 /
"""

with open("prod1.in.start", "w") as f:
    f.write(amd_content)

print("✅ prod1.in.start file written successfully:")
print("="*40)
print(amd_content)


✅ prod1.in.start file written successfully:
 MD explicit solvent heavy atom no rest shake dt 0.002 total is 100 nanoseconds
 &cntrl
  imin = 0, nstlim = 50000000, dt=0.002,
  ntx = 1, irest = 0,
  ntwx = 5000, ioutfm = 1, ntxo = 2, ntpr = 10000, ntwr = 500000,
  iwrap = 1, nscm = 1000,
  ntc = 2, ntf = 2, ntb = 2, cut = 8.0,
  ntt = 3, ig = -1, gamma_ln = 3, temp0 = 310, tempi = 310,
  ntp = 1, taup = 2.0, barostat = 2,
  igb = 0, saltcon = 0.0,
  ntr = 0,
  iamd=3,
  ethreshd=4698.75, alphad=220.0,
  ethreshp=-128637.3125, alphap=7350.400000000001,
 /



In [57]:
# Note that Ive changed dt to 0.002 and nstlim = 50000000; total time = 100ns still. Did this since 0.004 was unstable with aMD
#changed ntwx = 5000 (prints co-ordinates every 10ps, ntpr = 10000 - writes energy every 20ps and ntwr = 500000 - writes restart file every 1ns

amd_content = f""" MD explicit solvent heavy atom no rest shake dt 0.002 total is 100 nanoseconds
 &cntrl
  imin = 0, nstlim = 50000000, dt=0.002,
  ntx = 5, irest = 1,
  ntwx = 5000, ioutfm = 1, ntxo = 2, ntpr = 10000, ntwr = 500000,
  iwrap = 1, nscm = 1000,
  ntc = 2, ntf = 2, ntb = 2, cut = 8.0,
  ntt = 3, ig = -1, gamma_ln = 3, temp0 = 310, tempi = 310,
  ntp = 1, taup = 2.0, barostat = 2,
  igb = 0, saltcon = 0.0,
  ntr = 0,
  iamd=3,
  ethreshd={ETHRESHD}, alphad={ALPHAD},
  ethreshp={ETHRESHP}, alphap={ALPHAP},
 /
"""

with open("prod1.in", "w") as f:
    f.write(amd_content)

print("✅ prod1.in file written successfully:")
print("="*40)
print(amd_content)


✅ prod1.in file written successfully:
 MD explicit solvent heavy atom no rest shake dt 0.002 total is 100 nanoseconds
 &cntrl
  imin = 0, nstlim = 50000000, dt=0.002,
  ntx = 5, irest = 1,
  ntwx = 5000, ioutfm = 1, ntxo = 2, ntpr = 10000, ntwr = 500000,
  iwrap = 1, nscm = 1000,
  ntc = 2, ntf = 2, ntb = 2, cut = 8.0,
  ntt = 3, ig = -1, gamma_ln = 3, temp0 = 310, tempi = 310,
  ntp = 1, taup = 2.0, barostat = 2,
  igb = 0, saltcon = 0.0,
  ntr = 0,
  iamd=3,
  ethreshd=4698.75, alphad=220.0,
  ethreshp=-128637.3125, alphap=7350.400000000001,
 /



In [50]:
#Testing to see if it works 
!pmemd.cuda -O -i prod1.in.start -c ../../Minimization/step9.rst7 -p ../../Minimization/prepinHMR.prmtop -o PTR_001.out -x md_prod1_001.nc -r md_prod1_001.rst -inf md_prod1.mdinfo

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
^C


In [55]:
#Note dt had to be reduced to smaller time steps, dt = 0.004 was too aggresive changed it to 0.002 and it works fine